In [ ]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./type.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_manufacturer = df['Manufacturer'].tolist()
list_type = df['Type'].tolist()
list_url = df['Url'].tolist()

print('总数量：' + str(len(list_url)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import requests

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for manufacturer, type_, url in zip(list_manufacturer, list_type, list_url):
    work.put_nowait((manufacturer, type_, url))

def crawler():
    global df
    
    while not work.empty():
        manufacturer, type_, url = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(101):
            try:
                resp = requests.get(url,
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))
                
                no_parts_found = html.xpath('//span[contains(@class, "navlabellink") and contains(@class, "nvoffset") and contains(@class, "nnormal")]/text()')

                # = = = = = = = = = = = = = = =
                
                if len(no_parts_found) != 0 and no_parts_found[0] == 'No Parts Found':
                    df_temp = pd.DataFrame([{'status': 'ok',
                                             'Url': url,
                                             'Amount': 0}])
                    
                    status = 'ok'
                    
                    break
                
                # = = = = = = = = = = = = = = =
                
                title = html.xpath('//div[contains(@style, "display: inline-block; padding: 0em 5em 0em 5em;")]/text()')[0]

                # = = = = = = = = = = = = = = =
                
                if manufacturer not in title or type_ not in title or 'Part Numbers' not in title:
                    continue

                # = = = = = = = = = = = = = = =
                
                list_part = html.xpath('//a[@class="navlabellink nvoffset nnormal"]')
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'status': 'ok',
                                         'Url': url,
                                         'Amount': len(list_part)}])
                
                status = 'ok'
                
                break
                
            except:
                time.sleep(1)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Url': url}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(url +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(3):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./amount_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')